In order to run this notebook, you must have generated data with random dropout of trace vertices. This can be done by uncommenting the bottom of `generate_trace_data.ipynb`

In [ ]:
from pathlib import Path
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
from tqdm import tqdm

In [ ]:
max_id = 300


def classify_height(row):
    height = row["height"]
    if height <= 2:
        return height
    else:
        return 3


def numerical_class(row):
    _class = row["class"]
    if _class == "primary":
        return 0
    if _class == "collateral":
        return 1
    if _class == "terminal":
        return 2

In [ ]:
# specify brain1 or brain2 below
brain = "brain1"

root_dir = Path(os.path.abspath("")).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")
data_dir = os.path.join(experiment_dir, "data", brain)
segments_swc_dir = os.path.join(data_dir, "segments_swc")
print(f"Directory where swcs reside: {segments_swc_dir}")

In [ ]:
for i in range(20):
    print(f"****************Iter: {i}********************")
    dropout_version = "10pct_iter" + str(i)
    trace_data_dir = os.path.join(data_dir, "trace_data")
    trace_data_dir = os.path.join(trace_data_dir, "1")
    trace_data_dir = os.path.join(trace_data_dir, dropout_version)
    df_path = os.path.join(trace_data_dir, "df.csv")
    if os.path.exists(df_path):
        df = pd.read_csv(df_path)
    else:
        df = pd.DataFrame(
            columns=[
                "seg_id",
                "class",
                "height",
                "log_seg_length",
                "measure",
                "value",
                "log_value",
            ]
        )
        for i in tqdm(np.arange(0, max_id)):
            i = int(i)
            trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
            if os.path.exists(trace_data_path) is True:
                trace_data = np.load(trace_data_path, allow_pickle=True)

                for node in trace_data:
                    seg_length = node["seg_length"]
                    height = node["height"]
                    _class = node["class"]
                    mean_curvature = node["mean_curvature"]
                    mean_torsion = node["mean_torsion"]

                    log_seg_length = np.log10(seg_length)

                    log_mean_curvature = np.log10(mean_curvature)
                    df = df.append(
                        {
                            "seg_id": i,
                            "height": height,
                            "class": _class,
                            "log_seg_length": log_seg_length,
                            "measure": "curvature",
                            "value": mean_curvature,
                            "log_value": log_mean_curvature,
                        },
                        ignore_index=True,
                    )

                    log_mean_torsion = np.log10(mean_torsion)
                    df = df.append(
                        {
                            "seg_id": i,
                            "height": height,
                            "class": _class,
                            "log_seg_length": log_seg_length,
                            "measure": "torsion",
                            "value": mean_torsion,
                            "log_value": log_mean_torsion,
                        },
                        ignore_index=True,
                    )
        df.to_csv(df_path)
    df["class"] = df.apply(numerical_class, axis=1)
    df["height_class"] = df.apply(classify_height, axis=1)

    measures = ["curvature", "torsion"]

    classes = {0: "primary", 1: "collateral", 2: "terminal"}
    n = len(classes)
    matrix_pairs = np.triu(np.ones((n, n)), k=1)
    (coord_pairs_x, coord_pairs_y) = np.where(matrix_pairs == 1)

    for measure in measures:
        for class_1, class_2 in zip(coord_pairs_x, coord_pairs_y):
            Y1 = []
            Y2 = []
            segment_numbers = []
            X_class = []
            neuron_id = 0
            # collect data from all neurons
            for i in np.arange(0, max_id):
                sample_query = df.loc[
                    (df["seg_id"] == i)
                    & ((df["class"] == class_1))
                    & (df["measure"] == measure)
                ]
                num_segments = len(sample_query.index)
                if num_segments > 0:
                    Y1.append(np.mean(sample_query["value"].to_numpy()))
                    sample_query = df.loc[
                        (df["seg_id"] == i)
                        & ((df["class"] == class_2))
                        & (df["measure"] == measure)
                    ]
                    Y2.append(np.mean(sample_query["value"].to_numpy()))

            # Sign Test
            dif = np.subtract(Y1, Y2)
            k = np.sum(dif > 0)
            n = dif.shape[0]
            upper_bool = k > n // 2
            if upper_bool:
                p = np.sum(scipy.stats.binom.pmf(range(k, n + 1), n=n, p=0.5))
            else:
                p = np.sum(scipy.stats.binom.pmf(range(0, k + 1), n=n, p=0.5))

            if p < 0.05 / 12:
                if upper_bool:
                    symb = ">"
                else:
                    symb = "<"
                print(f"{classes[class_1]} {symb} {classes[class_2]} in {measure}")
                print(f"p-val was: {p}")
            else:
                print(
                    f"No difference between {classes[class_1]} and {classes[class_2]} in {measure}"
                )
                print(f"p-val was: {p}")